In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from tqdm.notebook import tqdm_notebook

In [2]:
title_basic_url = "https://datasets.imdbws.com/title.basics.tsv.gz"
title_aka_url = "https://datasets.imdbws.com/title.akas.tsv.gz"
title_rating_url = "https://datasets.imdbws.com/title.ratings.tsv.gz"

In [3]:
basics = pd.read_csv(title_basic_url, sep='\t', low_memory=False)

In [4]:
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...
8979341,tt9916848,tvEpisode,Episode #3.17,Episode #3.17,0,2010,\N,\N,"Action,Drama,Family"
8979342,tt9916850,tvEpisode,Episode #3.19,Episode #3.19,0,2010,\N,\N,"Action,Drama,Family"
8979343,tt9916852,tvEpisode,Episode #3.20,Episode #3.20,0,2010,\N,\N,"Action,Drama,Family"
8979344,tt9916856,short,The Wind,The Wind,0,2015,\N,27,Short


In [5]:
basics.replace({'\\N':np.nan},inplace=True)

In [6]:
basics_df = basics.dropna(subset = ['runtimeMinutes', 'genres', 'startYear'])

In [7]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2315073 entries, 0 to 8979345
Data columns (total 9 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   tconst          object
 1   titleType       object
 2   primaryTitle    object
 3   originalTitle   object
 4   isAdult         object
 5   startYear       object
 6   endYear         object
 7   runtimeMinutes  object
 8   genres          object
dtypes: object(9)
memory usage: 176.6+ MB


In [8]:
movie_filter = basics_df['titleType'] == 'movie'


In [9]:
basics_df = basics_df[movie_filter]

In [10]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 356207 entries, 570 to 8979296
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          356207 non-null  object
 1   titleType       356207 non-null  object
 2   primaryTitle    356207 non-null  object
 3   originalTitle   356207 non-null  object
 4   isAdult         356207 non-null  object
 5   startYear       356207 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  356207 non-null  object
 8   genres          356207 non-null  object
dtypes: object(9)
memory usage: 27.2+ MB


In [11]:
import os
os.makedirs('Data/', exist_ok=True)
os.listdir("Data/")

['aka.csv.gz',
 'final_tmdb_data_2000.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'movies.sql',
 'rating.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2000.json',
 'tmdb_api_results_2001.json']

In [12]:
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip',index=False)

In [13]:
del basics_df

In [14]:
aka = pd.read_csv(title_aka_url, sep='\t', low_memory=False)

In [15]:
aka

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0
...,...,...,...,...,...,...,...,...
32244327,tt9916852,5,Episódio #3.20,PT,pt,\N,\N,0
32244328,tt9916852,6,Episodio #3.20,IT,it,\N,\N,0
32244329,tt9916852,7,एपिसोड #3.20,IN,hi,\N,\N,0
32244330,tt9916856,1,The Wind,DE,\N,imdbDisplay,\N,0


In [16]:
aka_df = aka.replace({'\\N':np.nan})

In [17]:
is_US = aka_df['region'] == 'US'

In [18]:
aka_df = aka_df[is_US]

In [19]:
aka_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [20]:
#checking data info before saving
aka_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1326857 entries, 5 to 32244076
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1326857 non-null  object
 1   ordering         1326857 non-null  int64 
 2   title            1326857 non-null  object
 3   region           1326857 non-null  object
 4   language         3570 non-null     object
 5   types            1025868 non-null  object
 6   attributes       44141 non-null    object
 7   isOriginalTitle  1325482 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.1+ MB


In [21]:
aka_df.to_csv("Data/aka.csv.gz", compression='gzip',index=False)

In [22]:
#del aka_df

In [23]:
rating = pd.read_csv(title_rating_url, sep='\t', low_memory=False)

In [24]:
rating_df = rating.replace({'\\N':np.nan})

In [25]:
#checking data info before saving
rating_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1251880 entries, 0 to 1251879
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1251880 non-null  object 
 1   averageRating  1251880 non-null  float64
 2   numVotes       1251880 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [26]:
rating_df.to_csv("Data/rating.csv.gz", compression='gzip',index=False)

In [27]:
#Open basics_df
basics_df = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics_df.head()


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
1,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
2,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
3,tt0001184,movie,Don Juan de Serrallonga,Don Juan de Serrallonga,0,1910,NaN,58,"Adventure,Drama"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"


In [28]:
#Filter to only include US titles
us_filter = basics_df['tconst'].isin(aka_df['titleId'])

basics_df= basics_df[us_filter]
basics_df


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,NaN,70,"Action,Adventure,Biography"
1,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,NaN,90,Drama
2,tt0000679,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,NaN,120,"Adventure,Fantasy"
4,tt0001285,movie,The Life of Moses,The Life of Moses,0,1909,NaN,50,"Biography,Drama,Family"
5,tt0001498,movie,The Battle of Trafalgar,The Battle of Trafalgar,0,1911,NaN,51,War
...,...,...,...,...,...,...,...,...,...
356190,tt9915436,movie,Vida em Movimento,Vida em Movimento,0,2019,NaN,70,Documentary
356191,tt9915872,movie,The Last White Witch,My Girlfriend is a Wizard,0,2019,NaN,97,"Comedy,Drama,Fantasy"
356198,tt9916170,movie,The Rehearsal,O Ensaio,0,2019,NaN,51,Drama
356200,tt9916190,movie,Safeguard,Safeguard,0,2020,NaN,90,"Action,Adventure,Thriller"


In [29]:
#removing any movies classified as documentaries in the genre field
is_documentary = basics_df['genres'].str.contains('documentary', case = False)
basics_df = basics_df[~is_documentary]


In [30]:
#checking data info before saving
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149065 entries, 0 to 356202
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   tconst          149065 non-null  object 
 1   titleType       149065 non-null  object 
 2   primaryTitle    149065 non-null  object 
 3   originalTitle   149065 non-null  object 
 4   isAdult         149065 non-null  int64  
 5   startYear       149065 non-null  int64  
 6   endYear         0 non-null       float64
 7   runtimeMinutes  149065 non-null  int64  
 8   genres          149065 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 11.4+ MB


In [31]:
#Keeping start year after 2000-2021
basics_df['startYear']= basics_df['startYear'].astype(int)
basics_df = basics_df.loc[(basics_df['startYear'] >= 2000) & (basics_df['startYear'] <=2021)]

#Checking data before saving
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 79345 entries, 18264 to 356202
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          79345 non-null  object 
 1   titleType       79345 non-null  object 
 2   primaryTitle    79345 non-null  object 
 3   originalTitle   79345 non-null  object 
 4   isAdult         79345 non-null  int64  
 5   startYear       79345 non-null  int32  
 6   endYear         0 non-null      float64
 7   runtimeMinutes  79345 non-null  int64  
 8   genres          79345 non-null  object 
dtypes: float64(1), int32(1), int64(2), object(5)
memory usage: 5.8+ MB


C:\Users\RJ\anaconda3\envs\dojo-env\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [32]:
basics_df.to_csv("Data/title_basics.csv.gz", compression='gzip',index=False)

In [ ]:
!pip install tmdbsimple
import tmdbsimple as tmdb

In [ ]:
with open('C:/Users/RJ/.secret/api.json') as f:
    login = json.load(f)
login.keys()

In [ ]:
tmdb.API_KEY = login['api-key']

In [ ]:
#Testing with The Matrix
movie = tmdb.Movies(603)

In [ ]:
movie.info()

In [ ]:
# Function to obtain movie rating
def get_movieinfo_with_rating(movie_id):
    # Get movie and release dates
    movie = tmdb.Movies(movie_id)
    # Construct output dictionary
    movie_info = movie.info()
    releases = movie.releases()
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
            movie_info['certification'] = c['certification']
    return movie_info

In [ ]:
get_movieinfo_with_rating(603)

In [ ]:
import os
FOLDER = "DATA/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

In [ ]:
YEARS_TO_GET = [2000,2001]

In [ ]:
# Load in the IMDB title_basics file
basics = pd.read_csv("Data/title_basics.csv.gz")



In [ ]:
# Append new results to the existing JSON file
# Adapted from: 
# https://www.geeksforgeeks.org/append-to-json-file-using-python/   

def write_json(new_data, filename): 
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [ ]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)


In [ ]:
for YEAR in tqdm_notebook(YEARS_TO_GET, desc='YEARS', position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    if file_exists == False:
    
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
               
    df = basics.loc[ basics['startYear']==YEAR].copy()
    movie_ids = df['tconst'].copy()
    previous_df = pd.read_json(JSON_FILE)

    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]


    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        
        try:
            temp = get_movieinfo_with_rating(movie_id)
            write_json(temp,JSON_FILE)
            # Pause 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        
        except Exception as e:
            continue
            
    # Save the year's results as csv.gz file
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", 
                         compression="gzip", index=False)


